In [103]:
#this is a test

import pandas as pd
from util.db_util import Database

def get_points(df, values, scores):
    """
    Updates the scores dictionary in place.
    Handles:
    - Ghost scoring (scoring places shift upward)
    - Ties: athletes tied receive the average of the scoring places they occupy
    """

    for event in df["event"].unique():
        # Load event entries
        event_df = df[df["event"] == event].copy()
        event_df.sort_values("place", inplace=True)
        event_df["place"] = event_df["place"].astype(int)

        scoring_place = 1  # ghost scoring place
        i = 0
        n = len(event_df)

        while i < n:
            actual_place = event_df.iloc[i]["place"]

            # Identify complete tie group
            tie_group = event_df[event_df["place"] == actual_place]
            tie_size = len(tie_group)
            tie_indices = tie_group.index.tolist()

            # Number of athletes in tie = tie_size
            scoring_slots = []
            for k in range(tie_size):
                sp = scoring_place + k
                if sp in values:
                    scoring_slots.append(sp)

            # Compute point average if scoring slots exist
            pts_value = 0
            if scoring_slots:
                pts_value = sum(values[p] for p in scoring_slots) / tie_size

            # Assign points to all athletes in tie group
            for idx in tie_indices:
                school = event_df.loc[idx, "school_id"]
                scores[school] = scores.get(school, 0) + pts_value

            # Advance scoring place by tie size
            scoring_place += tie_size
            i += tie_size


db = Database("db/Track.db")

meet_type = input("Enter meet type (e.g. Sectional, Regional, State): ")

meet_number = 1
if(meet_type != "State"):
    meet_number = input("Enter meet number (e.g. 4): ") # enter in a number 1-32 for Sectional and 1-8 for Regional

year = input("Enter year (e.g. 2024): ")
gender = input("Enter gender (e.g. Girls or Boys): ")
scoring_type = int(input("Type 1 for Normal scoring and 2 for Enrollment range scoring: "))

low = 0
high = 10000

# Ex: if low is 200 and high is 1500, it would be get all entries of athletes with schools between 200-1500 enrollment size
if(scoring_type == 2):
    low = input("Enter lower bound of enrollment size (inclusive and e.g. 809): ") 
    high = input("Enter upper bound of enrollment size (inclusive and e.g. 8000): ")

# dictionary that has point values for state meet scoring
# can modify it if you want to test out different scoring systems, such as changing point values or going through top 20 athletes
state_values = {
    1: 10,
    2: 8,
    3: 7,
    4: 6,
    5: 5,
    6: 4,
    7: 3,
    8: 2,
    9: 1,
}

sec_reg_values = {
    1: 10,
    2: 8,
    3: 6,
    4: 5,
    5: 4,
    6: 3,
    7: 2,
    8: 1,
}

values = { }

# assign the "current" dictionary with certain values based on which meet type is selected
if(meet_type != "State"):
    values = sec_reg_values
else:
    values = state_values

top_num = len(values) # gets all athletes that will be scored for a certain meet... auto updates if we want to test scoring more athletes
    
meet_num = db.get_meet_id(meet_type, meet_number, year, gender)

individual_df = db.get_top_results_from_meet(top_num, meet_num, "Final", low, high)

relay_df = db.get_top_relays_from_meet(top_num, meet_num, low, high)

scores = { }

get_points(individual_df, values, scores)
get_points(relay_df, values, scores)

scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)) # sorts schools based on points

pd.set_option('display.max_rows', None)

# conversion into a dataframe for formatting so the output looks a bit cleaner
sorted_df = pd.DataFrame(list(scores.items()), columns=["School", "Score"])
sorted_df["Place"] = sorted_df["Score"].rank(method="min", ascending=False).astype(int)
sorted_df["School"] = sorted_df["School"].apply(db.get_school_name)
sorted_df["Score"] = sorted_df["Score"].round(2)
sorted_df.set_index("Place", inplace=True)
sorted_df

Enter meet type (e.g. Sectional, Regional, State):  Regional
Enter meet number (e.g. 4):  4
Enter year (e.g. 2024):  2025
Enter gender (e.g. Girls or Boys):  Boys
Type 1 for Normal scoring and 2 for Enrollment range scoring:  1


,School,Score
Place,,
1,Noblesville,84.0
2,Hamilton Southeastern,69.0
3,Fishers,67.0
4,Westfield,66.0
5,Harrison (West Lafayette),52.0
6,Carmel,51.0
7,Lebanon,38.0
8,Kokomo,36.5
9,McCutcheon,21.0
